In [1]:

import pandas as pd
import sys 
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import _tree
import numpy as np
import pickle
from sklearn.decomposition import PCA
from scipy import stats
from scipy.stats import norm
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pandas as pd 
import math
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import random as rd
import sys
import time
import statistics

In [2]:
sys.path.append('../')
import pcsp
from pcsp import PCSPipeline, ModuleSet, Module, init_args # must install pcsp first (pip install pcsp)
from pcsp.pipeline import build_graph

In [3]:
sys.path.append("../../correlated_irf/iterative-Random-Forest/")
from irf import irf_utils
from irf.ensemble import RandomForestClassifierWithWeights,RandomForestRegressorWithWeights

# Load Data 

In [11]:
X_train = pd.read_csv("../data/01_X_train.csv",error_bad_lines=False).iloc[:,1:]
X_test = pd.read_csv("../data/02_X_test.csv",error_bad_lines=False).iloc[:,1:]
y_train = pd.read_csv("../data/03_y_train.csv",error_bad_lines=False)
y_test = pd.read_csv("../data/04_y_test.csv",error_bad_lines=False)

In [12]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,13.0,12.0,14.0,17.0,12.0,10.0,15.0,4.0,19.0,13.0,...,0.0,0.0,0.00000,0.0000,0.00000,1.28717,0.00000,1.18738,1.33880,0.00000
1,10.0,14.0,16.0,14.0,68.0,19.0,14.0,27.0,60.0,25.0,...,0.0,0.0,0.00000,0.0000,0.00000,0.00000,0.00000,2.36356,2.17012,0.00000
2,21.0,20.0,29.0,15.0,15.0,23.0,27.0,31.0,32.0,39.0,...,0.0,0.0,1.96269,0.0000,0.00000,1.28678,2.07168,3.11724,3.16796,1.33514
3,14.0,22.0,19.0,18.0,19.0,6.0,17.0,13.0,26.0,15.0,...,0.0,0.0,0.00000,0.0000,0.00000,0.00000,0.00000,6.51908,6.79381,1.16224
4,21.0,23.0,27.0,20.0,18.0,11.0,10.0,6.0,8.0,10.0,...,0.0,0.0,0.00000,0.0000,0.00000,1.19790,0.00000,0.00000,0.00000,1.19955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3907,13.0,16.0,21.0,14.0,13.0,21.0,8.0,8.0,20.0,13.0,...,0.0,0.0,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3908,12.0,11.0,16.0,14.0,22.0,16.0,17.0,1.0,30.0,22.0,...,0.0,0.0,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3909,10.0,8.0,9.0,11.0,29.0,26.0,33.0,12.0,48.0,49.0,...,0.0,0.0,0.00000,2.1888,1.38237,2.23157,2.50635,6.55166,6.84249,0.00000
3910,10.0,9.0,14.0,17.0,13.0,26.0,25.0,10.0,15.0,15.0,...,0.0,0.0,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,1.24466


# run iRF 

In [7]:
n_samples = 1000
n_features = 10
X_train = np.random.uniform(low=0, high=1, size=(n_samples, n_features))
y_train = np.random.choice([0, 1], size=(n_samples,), p=[.5, .5])
X_test = np.random.uniform(low=0, high=1, size=(n_samples, n_features))
y_test = np.random.choice([0, 1], size=(n_samples,), p=[.5, .5])
# The second feature (which is indexed by 1) is very important
X_train[:, 1] = X_train[:, 1] + y_train
X_test[:, 1] = X_test[:, 1] + y_test

In [ ]:
all_rf_weights, all_K_iter_rf_data, \
    all_rf_bootstrap_output, all_rit_bootstrap_output, \
    stability_score = irf_utils.run_iRF(X_train=np.array(X_train),
                                        X_test=np.array(X_test),
                                        y_train=np.array(y_train),
                                        y_test=np.array(y_test),
                                        K=5,                          # number of iteration
                                        rf = RandomForestRegressorWithWeights(n_estimators=20),
                                        B=30,
                                        random_state_classifier=2018, # random seed
                                        propn_n_samples=.2,
                                        bin_class_type=1,
                                        M=20,
                                        max_depth=5,
                                        noisy_split=False,
                                        num_splits=2,
                                        n_estimators_bootstrap=5)

In [9]:
print(all_rf_weights['rf_weight5'])


[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
print(stability_score)


{'1': 1.0}
